In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("trip_count_by_zone_sql").getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/01/06 18:50:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/01/06 18:50:46 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/01/06 18:50:46 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [6]:
directory = "/Users/junghee/data-engineering/01-spark/data"
trip_file = "/fhvhv_tripdata_2020-03.csv"
zone_file = "taxi+_zone_lookup.csv"

In [10]:
trip_data = spark.read.csv(f"file:///{directory}/{trip_file}", inferSchema = True, header = True)
zone_data = spark.read.csv(f"file:///{directory}/{zone_file}", inferSchema = True, header = True)

In [12]:
zone_data.show(5)

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
+----------+-------------+--------------------+------------+
only showing top 5 rows



In [13]:
trip_data.createOrReplaceTempView("trip")
zone_data.createOrReplaceTempView("zone")

In [15]:
spark.sql("select * from zone").show(5)

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
+----------+-------------+--------------------+------------+
only showing top 5 rows



In [16]:
spark.sql("select * from trip").show(5)

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0005|              B02510|2020-03-01 00:03:40|2020-03-01 00:23:39|          81|         159|   null|
|           HV0005|              B02510|2020-03-01 00:28:05|2020-03-01 00:38:57|         168|         119|   null|
|           HV0003|              B02764|2020-03-01 00:03:07|2020-03-01 00:15:04|         137|         209|      1|
|           HV0003|              B02764|2020-03-01 00:18:42|2020-03-01 00:38:42|         209|          80|   null|
|           HV0003|              B02764|2020-03-01 00:44:24|2020-03-01 00:58:44|         256|         226|   null|
+-----------------+--------------------+-------------------+-------------------+

In [24]:
spark.sql("select Borough, count(*) as trip \
            from (select Borough \
                        from trip join zone \
                        on trip.PULocationID = zone.LocationID)\
            group by Borough\
            order by trip desc").show()

+-------------+-------+
|      Borough|   trip|
+-------------+-------+
|    Manhattan|4953140|
|     Brooklyn|3735764|
|       Queens|2437383|
|        Bronx|2086592|
|Staten Island| 178818|
|      Unknown|    845|
|          EWR|    362|
+-------------+-------+



In [27]:
spark.sql("select Borough, count(*) as trip \
            from (select Borough \
                        from trip join zone \
                        on trip.DOLocationID = zone.LocationID)\
            group by Borough\
            order by trip desc").explain(True)

== Parsed Logical Plan ==
'Sort ['trip DESC NULLS LAST], true
+- 'Aggregate ['Borough], ['Borough, 'count(1) AS trip#480]
   +- 'SubqueryAlias __auto_generated_subquery_name
      +- 'Project ['Borough]
         +- 'Join Inner, ('trip.DOLocationID = 'zone.LocationID)
            :- 'UnresolvedRelation [trip], [], false
            +- 'UnresolvedRelation [zone], [], false

== Analyzed Logical Plan ==
Borough: string, trip: bigint
Sort [trip#480L DESC NULLS LAST], true
+- Aggregate [Borough#176], [Borough#176, count(1) AS trip#480L]
   +- SubqueryAlias __auto_generated_subquery_name
      +- Project [Borough#176]
         +- Join Inner, (DOLocationID#150 = LocationID#175)
            :- SubqueryAlias trip
            :  +- View (`trip`, [hvfhs_license_num#145,dispatching_base_num#146,pickup_datetime#147,dropoff_datetime#148,PULocationID#149,DOLocationID#150,SR_Flag#151])
            :     +- Relation [hvfhs_license_num#145,dispatching_base_num#146,pickup_datetime#147,dropoff_datetime#148